In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [10]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [11]:
base_csv = './data/IMDB Dataset.csv'
df = pd.read_csv(base_csv)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

In [13]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Download NLTK stop words (if not already downloaded)
nltk.download('stopwords')
# Preprocess the text to extract the vocabulary
def preprocess_text(text):
    text = text.lower()
    text = abbreviations[text.lower()] if text.lower() in abbreviations.keys() else text
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.strip()
    stop_words = set(stopwords.words('english'))  
    text_tokens = text.split()  
    filtered_text = [word for word in text_tokens if word not in stop_words]
    text = ' '.join(filtered_text)
    return text

# Tokenize and build the vocabulary
def build_vocab(dataframe, column_name):
    word_list = []
    for review in dataframe[column_name]:
        processed_review = preprocess_text(review)
        word_list.extend(processed_review.split())
    # Count word frequencies
    word_counts = Counter(word_list)
    return word_counts


vocab = build_vocab(df, 'review')
print(f"Vocabulary size: {len(vocab)}")

[nltk_data] Downloading package stopwords to /Users/jack/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Vocabulary size: 221464


In [14]:
# Create a word-to-index dictionary
word_to_index = {word: i+1 for i, word in enumerate(vocab.keys())}  # Start indexing from 1
print(f"Word to index mapping: {list(word_to_index.items())[:10]}")  # Show the first 10 mappings

Word to index mapping: [('one', 1), ('reviewers', 2), ('mentioned', 3), ('watching', 4), ('1', 5), ('oz', 6), ('episode', 7), ('youll', 8), ('hooked', 9), ('right', 10)]


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert reviews to sequences of integers
def text_to_sequence(text, word_to_index):
    processed_text = preprocess_text(text)
    return [word_to_index[word] for word in processed_text.split() if word in word_to_index]

# Tokenize the reviews
df['review_sequences'] = df['review'].apply(lambda x: text_to_sequence(x, word_to_index))
df['label_convert'] = df['sentiment'].apply(lambda x: 1 if x =='positive' else 0)

In [42]:
# Train-test split with shuffle
X_train, y_train, X_test, y_test = train_test_split(df['review_sequences'], df['label_convert'], test_size=0.2, random_state=42, shuffle=True)
# Pad the sequences to a fixed length
max_length = 100  # Maximum sequence length
x_train_pad = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
y_train_pad = pad_sequences(y_train, maxlen=max_length, padding='post', truncating='post')

print(f"Padded sequences shape: {x_train_pad.shape}")
print("Padded Array:", x_train_pad)
print("Padded Array:", y_train_pad)

Padded sequences shape: (40000, 100)
Padded Array: [[  626  1251  3129 ...   214 27947 10648]
 [  336   158   295 ...   919  3196  2333]
 [ 4438   332   420 ...     0     0     0]
 ...
 [ 2207     1  1167 ...     0     0     0]
 [  146   878  1069 ...     0     0     0]
 [ 3266   295  1204 ...     0     0     0]]
Padded Array: [[   186    607  84411 ...    239   1456 106497]
 [   251   2249     88 ...   2120   3037   4395]
 [   306   4404    380 ...      0      0      0]
 ...
 [   747  63135    565 ...     81   2623   7873]
 [    79    573     23 ...      0      0      0]
 [   103   4795    259 ...      0      0      0]]


In [43]:
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(X_test.values))
valid_data = TensorDataset(torch.from_numpy(y_train_pad), torch.from_numpy(y_test.values))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [44]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)

Sample input size:  torch.Size([50, 100])
Sample input: 
 tensor([[ 19106,    469,    185,  ...,    102,   1376,    169],
        [   616,   2613,    184,  ...,      0,      0,      0],
        [   544,    217,   4240,  ...,      0,      0,      0],
        ...,
        [121817,  10481,  22056,  ...,   4468,    226,  31332],
        [  2487,    651,    295,  ...,    215, 171415,  59820],
        [ 19352,   3878,    295,  ...,    152,    515,   7111]],
       dtype=torch.int32)
Sample input: 
 tensor([1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1])


In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Hyperparameters
VOCAB_SIZE = len(vocab) + 1          # Vocabulary size (e.g., IMDB dataset)
EMBEDDING_DIM = 100         # Embedding dimension
HIDDEN_DIM = 50            # Hidden size for RNN
OUTPUT_DIM = 1              # Output dimension (binary classification)
N_LAYERS = 1                # Number of RNN layers
DROPOUT = 0.5               # Dropout probability
LEARNING_RATE = 0.001       # Learning rate
NUM_EPOCHS = 10             # Number of epochs
BATCH_SIZE = 50             # Batch size
MAX_LENGTH = 100            # Maximum sequence length
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available

# Encoder (Vanilla RNN)
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, dropout):
        super(EncoderRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # RNN
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=n_layers, dropout=dropout, batch_first=True)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input_seq):
        # input_seq: [batch_size, seq_len]
        embedded = self.dropout(self.embedding(input_seq))  # [batch_size, seq_len, embedding_dim]
        outputs, hidden = self.rnn(embedded)                # outputs: [batch_size, seq_len, hidden_dim]
        return hidden[-1]                                   # Return the final hidden state (h_T)

# Decoder (Classification Layer)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(DecoderRNN, self).__init__()
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        # Sigmoid for binary classification
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, hidden):
        # hidden: [batch_size, hidden_dim]
        output = self.fc(hidden)  # [batch_size, output_dim]
        return self.sigmoid(output)

# Combined Model (Encoder + Decoder)
class SentimentModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(SentimentModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, input_seq):
        # Pass through encoder
        encoder_hidden = self.encoder(input_seq)
        
        # Pass encoder's output to decoder
        output = self.decoder(encoder_hidden)
        return output

# Initialize model, loss function, and optimizer
encoder = EncoderRNN(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, N_LAYERS, DROPOUT).to(DEVICE)
decoder = DecoderRNN(HIDDEN_DIM, OUTPUT_DIM).to(DEVICE)
model = SentimentModel(encoder, decoder).to(DEVICE)

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training Function
def train_model(model, optimizer, criterion, train_loader, num_epochs):
    model.train()  # Set the model to training mode
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        
        for input_seq, labels in train_loader:
            # Move data to the appropriate device
            input_seq, labels = input_seq.to(DEVICE), labels.to(DEVICE)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Forward pass
            predictions = model(input_seq).squeeze(1)  # [batch_size]
            
            # Compute the loss
            loss = criterion(predictions, labels.float())
            
            # Backward pass
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
            # Accumulate loss
            epoch_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}')


# Train the Model
train_model(model, optimizer, criterion, train_loader, NUM_EPOCHS)

/opt/anaconda3/envs/FYP2/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
